In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go

# Load data
df_poi = pd.read_csv('../data/POI.csv')
dtype_spec = {
    'Timestamp': str,
    'id': int,
    'lat': float,
    'long': float,
    'Date': str,
    'Hour': int,
    'Minute': int,
    'Second': int,
    'LastName': str,
    'FirstName': str,
    'CurrentEmploymentType': str,
    'CurrentEmploymentTitle': str,
    'RouteID': int
}
df_route = pd.read_csv('../data/gpsWithNameandRoute.csv', dtype=dtype_spec)

# Define color mapping based on time of day
def get_color(hour):
    if 0 <= hour < 6:
        return 'blue'  # Midnight
    elif 6 <= hour < 12:
        return 'green'  # Morning
    elif 12 <= hour < 13:
        return 'yellow'  # Noon
    elif 13 <= hour < 18:
        return 'orange'  # Afternoon
    elif 18 <= hour < 21:
        return 'red'  # Evening
    else:
        return 'purple'  # Night

df_route['color'] = df_route['Hour'].apply(get_color)

# Define time period based on hour
def get_time_period(hour):
    if 0 <= hour < 6:
        return 'Midnight'
    elif 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 13:
        return 'Noon'
    elif 13 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df_route['time_period'] = df_route['Hour'].apply(get_time_period)

# Create Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("POI and Route Visualization"),
    
    html.Label('Select ID'),
    dcc.Dropdown(
        id='id-dropdown',
        options=[{'label': str(i), 'value': i} for i in df_route['id'].unique()],
        value=df_route['id'].unique()[0],
        clearable=False,
    ),
    
    html.Label('Select Route ID'),
    dcc.Dropdown(
        id='route-dropdown',
        clearable=False,
    ),
    
    dcc.Graph(id='map-graph')
])

@app.callback(
    Output('route-dropdown', 'options'),
    Output('route-dropdown', 'value'),
    Input('id-dropdown', 'value')
)
def set_route_options(selected_id):
    filtered_routes = df_route[df_route['id'] == selected_id]
    route_options = []
    for route_id in filtered_routes['RouteID'].unique():
        route_df = filtered_routes[filtered_routes['RouteID'] == route_id]
        date = route_df['Date'].iloc[0]
        time_period = route_df['time_period'].iloc[0]
        label = f"{route_id} - {date} {time_period}"
        route_options.append({'label': label, 'value': route_id})
    
    value = route_options[0]['value'] if len(route_options) > 0 else None
    return route_options, value

@app.callback(
    Output('map-graph', 'figure'),
    Input('id-dropdown', 'value'),
    Input('route-dropdown', 'value')
)
def update_figure(selected_id, selected_route_id):
    filtered_df = df_route[(df_route['id'] == selected_id) & (df_route['RouteID'] == selected_route_id)]
    
    fig = go.Figure()

    # Add POI data points
    fig.add_trace(go.Scatter(
        x=df_poi['long'], 
        y=df_poi['lat'], 
        mode='markers',
        text=df_poi['name'],
        textposition='top center',
        marker=dict(size=10),
        name='POI',
        hovertemplate='<b>%{text}</b><br>Latitude: %{y}<br>Longitude: %{x}<extra></extra>'
    ))

    # Add route data points and connect them with lines, color varies by time of day
    for color in filtered_df['color'].unique():
        route_segment = filtered_df[filtered_df['color'] == color]
        fig.add_trace(go.Scatter(
            x=route_segment['long'], 
            y=route_segment['lat'], 
            mode='lines+markers',
            line=dict(shape='linear', width=2, color=color),
            marker=dict(size=6, color=color),
            name=f'Route ({color})',
            hovertemplate=(
                'Timestamp: %{customdata[0]}<br>'
                'ID: %{customdata[1]}<br>'
                'Latitude: %{y}<br>'
                'Longitude: %{x}<br>'
                'Date: %{customdata[2]}<br>'
                'Hour: %{customdata[3]}<br>'
                'Minute: %{customdata[4]}<br>'
                'Second: %{customdata[5]}<br>'
                'LastName: %{customdata[6]}<br>'
                'FirstName: %{customdata[7]}<br>'
                'CurrentEmploymentType: %{customdata[8]}<br>'
                'CurrentEmploymentTitle: %{customdata[9]}<br>'
                'RouteID: %{customdata[10]}<extra></extra>'
            ),
            customdata=route_segment[['Timestamp', 'id', 'Date', 'Hour', 'Minute', 'Second', 
                                      'LastName', 'FirstName', 'CurrentEmploymentType', 
                                      'CurrentEmploymentTitle', 'RouteID']].values
        ))

    # Update layout to add color legend
    fig.update_layout(
        title='POI and Route Visualization',
        xaxis_title='Longitude',
        yaxis_title='Latitude',
        autosize=True,
        margin=dict(l=40, r=40, b=40, t=40),
        hovermode='closest',
        legend=dict(
            title="Time of Day",
            itemsizing='constant'
        )
    )

    # Add color legend
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, color='blue'),
        legendgroup='Midnight',
        showlegend=True,
        name='Midnight (00:00-06:00)'
    ))

    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, color='green'),
        legendgroup='Morning',
        showlegend=True,
        name='Morning (06:00-12:00)'
    ))

    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, color='yellow'),
        legendgroup='Noon',
        showlegend=True,
        name='Noon (12:00-13:00)'
    ))

    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, color='orange'),
        legendgroup='Afternoon',
        showlegend=True,
        name='Afternoon (13:00-18:00)'
    ))

    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, color='red'),
        legendgroup='Evening',
        showlegend=True,
        name='Evening (18:00-21:00)'
    ))

    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, color='purple'),
        legendgroup='Night',
        showlegend=True,
        name='Night (21:00-00:00)'
    ))

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8053)
